In [1]:
import pandas as pd
import os

In [10]:
ccodes = ['HU','SK','CZ','PL','PL_SK_CZ_HU_LPR']
vars = ['LPR','HER']
for dir_path, dirs, files in os.walk("/home/donggyu/RegionalEstimationLLM/data/label"):
    for file in files:
        if any(ccode in file for ccode in ccodes) and any(var in file for var in vars):
# file_path = "/home/donggyu/RegionalEstimationLLM/data/label/HU_LPR.csv"
            file_path = os.path.join(dir_path, file)
            df = pd.read_csv(file_path)
            df = df.round(1)
            
            df.to_csv(file_path,index=False)

In [4]:
import pandas as pd
from difflib import get_close_matches

# 예시 CSV 읽기 (실제로는 pd.read_csv 등으로 파일/문자열에서 읽으시면 됩니다.)
df1 = pd.read_csv("/home/donggyu/RegionalEstimationLLM/data/label/PL_GRDP.csv")
df2 = pd.read_csv("/home/donggyu/RegionalEstimationLLM/data/label/PL_ELP.csv")

#--------------------------------------------------------
# 1) df2의 area_name에서 "PODREGION " 같은 접두어는 제거(매칭 정확도 향상 목적)
# 2) difflib의 get_close_matches로 df1['area_name'] 중 가장 유사한 값을 찾는다.
# 3) 그 값이 있으면 df2의 area_name을 해당 값으로 덮어쓰고, df1에서 가져온 area_id를 df2에 넣는다.
#--------------------------------------------------------

# 1) 불필요한 접두어 제거(선택 사항)
df2['area_name_clean'] = df2['area_name'].str.replace('PODREGION ', '', regex=False).str.strip()

for idx in range(len(df2)):
    name_to_match = df2.loc[idx, 'area_name_clean']
    
    # df1에서 가장 유사도가 높은 area_name을 하나 가져옴
    best_match = get_close_matches(name_to_match, df1['area_name'], n=1, cutoff=0)
    
    if best_match:
        matched_name = best_match[0]
        # df2의 area_name을 df1의 area_name으로 교체
        df2.loc[idx, 'new_area_name'] = matched_name
        # df1에서 해당 area_name의 area_id 가져와서 df2의 area_id에 할당
        df2.loc[idx, 'new_area_id'] = df1.loc[df1['area_name'] == matched_name, 'area_id'].values[0]
    else:
        # 매칭되는 게 전혀 없다면(rare case), 그대로 둔다거나 예외 처리를 할 수 있음
        df2.loc[idx, 'area_name'] = name_to_match

# 최종적으로 필요 없다면 clean 컬럼 삭제
df2.drop(columns='area_name_clean', inplace=True)

# 결과 확인
print(df2)


                  area_name  area_id     ELP  new_area_name new_area_id
0       PODREGION KRAKOWSKI      NaN  114595        Słupski       PL636
1   PODREGION MIASTO KRAKÓW      NaN  155362  Miasto Kraków       PL213
2     PODREGION NOWOSĄDECKI      NaN   78618          Nyski       PL523
3     PODREGION OŚWIĘCIMSKI      NaN  100040  Miasto Kraków       PL213
4       PODREGION TARNOWSKI      NaN   77520          Tyski       PL22C
..                      ...      ...     ...            ...         ...
68     PODREGION OSTROŁĘCKI      NaN   62283          Tyski       PL22C
69       PODREGION RADOMSKI      NaN  107745  Miasto Kraków       PL213
70         PODREGION PŁOCKI      NaN   59274         Łódzki       PL712
71      PODREGION SIEDLECKI      NaN   72561          Ełcki       PL623
72    PODREGION ŻYRARDOWSKI      NaN   45893        Słupski       PL636

[73 rows x 5 columns]
